In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage import data
from skimage.filters import gaussian
from skimage.segmentation import active_contour
import skimage.io as io 
import scipy

In [3]:
x = ['x'+ str(x) for x in range(100)]
y = ['y'+ str(y) for y in range(100)]

columns = ['id'] + list(sum(zip(x, y), ()))

In [4]:
def interpcurve(N, pX, pY):
    #equally spaced in arclength
    N = np.transpose(np.linspace(0, 1, N))
    #how many points will be uniformly interpolated?
    nt = N.size

    #number of points on the curve
    n = pX.size
    pxy = np.array((pX, pY)).T
    p1 = pxy[0,:]
    pend = pxy[-1,:]
    last_segment = np.linalg.norm(np.subtract(p1, pend))
    epsilon= 10 * np.finfo(float).eps

    #IF the two end points are not close enough lets close the curve
    if last_segment > epsilon * np.linalg.norm(np.amax(abs(pxy), axis=0)):
        pxy = np.vstack((pxy, p1))
        nt = nt + 1

    pt = np.zeros((nt, 2))

    #Compute the chordal arclength of each segment.
    chordlen = (np.sum(np.diff(pxy, axis=0) ** 2, axis=1)) ** (1 / 2)
    #Normalize the arclengths to a unit total
    chordlen = chordlen / np.sum(chordlen)
    #cumulative arclength
    cumarc = np.append(0, np.cumsum(chordlen))

    tbins= np.digitize(N, cumarc) # bin index in which each N is in

    #catch any problems at the ends
    tbins[np.where(tbins<=0 | (N<=0))]=1
    tbins[np.where(tbins >= n | (N >= 1))] = n - 1      

    s = np.divide((N - cumarc[tbins]), chordlen[tbins-1])
    pt = pxy[tbins,:] + np.multiply((pxy[tbins,:] - pxy[tbins-1,:]), (np.vstack([s]*2)).T)

    return pt 

In [5]:
from skimage import measure
import pandas as pd
from os.path import basename
from skimage.color import rgb2gray

df_semilandmarks = pd.DataFrame(columns=columns)
collection = io.ImageCollection('output_data/clean/*.png')
print(len(collection))
counter = 0
for index, img in enumerate(collection):
    id_name = basename(collection.files[index])
    img = rgb2gray(img)
    img = img[:, ::-1]
    #print(img)
    img = np.pad(img, pad_width=6, mode='constant', constant_values=1.)
    #print(img)
    #print(img.shape)    
    if img.ndim == 2:
        contours = measure.find_contours(img, 0.1, fully_connected='high')
        counter+=1  
        #for n, contour in enumerate(contours):
        #print(contours[0].shape)
        contour = contours[0]
        #print(contour)
        if 115. in contour[:, 1]:
            #fig = plt.figure(figsize=(7, 7))
            #ax = fig.add_subplot(111)
            #ax.imshow(img, interpolation='nearest', cmap=plt.cm.gray)
            #ax.plot(contour[:, 1], contour[:, 0], linewidth=5) #, color='b')
        #else:
        #    ax.plot(contour[:, 1], contour[:, 0], linewidth=5, color='r')
        # resample_contour = contour[np.random.choice(contour.shape[0], 150, replace=False), :]
            resample_contour = interpcurve(100,  contour[:, 0],  contour[:, 1])
            # print(resample_contour[:4, 0], resample_contour[:4, 1], resample_contour[:4].ravel())
            df_semilandmarks.loc[index] = [id_name] + list(resample_contour.ravel())
            #ax.plot(resample_contour[:, 1], resample_contour[:, 0], 'om', linewidth=5)
        #plt.savefig('output/landmarked_'+id_name)
        #plt.show()
    # To only show the first three on this ipynb
    #if counter == 15:
    #    break

1140


In [6]:
df_semilandmarks

,id,x0,y0,x1,y1,x2,y2,x3,y3,x4,...,x95,y95,x96,y96,x97,y97,x98,y98,x99,y99
0,image_0.png,117.0,48.055556,115.097676,52.028037,112.518968,55.634014,110.046863,59.269680,107.337208,...,116.035708,30.325981,116.398219,34.806515,116.979567,39.222595,116.686315,43.632044,117.0,48.055556
1,image_00.png,116.0,39.602564,112.733716,40.336351,109.509279,41.035268,106.057990,40.828342,103.861624,...,110.368644,27.648585,112.127204,30.575238,114.323353,33.216189,115.791808,36.266419,116.0,39.602564
2,image_01.png,107.0,67.093750,105.590066,69.739199,104.551141,72.588288,103.172453,75.285509,102.169309,...,104.725353,68.976166,105.701253,66.125530,106.789952,63.277113,107.213906,63.988904,107.0,67.093750
3,image_1.png,116.0,51.142276,113.160288,55.180620,110.342777,59.532238,106.847712,63.410210,103.159228,...,114.127984,30.567664,115.688343,35.483474,115.998633,40.702551,116.125518,45.882724,116.0,51.142276
4,image_2.png,12.0,89.112903,11.603448,89.790879,11.565046,90.536647,11.528373,91.297038,11.265698,...,11.233948,86.179262,11.174424,86.962436,11.519081,87.662104,11.855503,88.371173,12.0,89.112903
5,image_02.png,116.0,56.143750,112.205074,56.049549,109.096454,54.180884,106.308764,51.797084,103.458449,...,110.907513,42.784301,113.500418,45.250565,115.128793,48.666675,116.159282,52.272130,116.0,56.143750
6,image_03.png,116.0,55.675000,112.961829,59.507222,109.500173,63.080097,107.645899,67.534446,106.042930,...,114.625006,35.975458,115.282725,40.849776,116.262887,45.676440,116.256189,50.662004,116.0,55.675000
7,image_4.png,115.0,58.573529,111.684637,60.894909,108.607523,63.608452,106.679319,67.170269,105.108139,...,114.660386,42.213611,115.101277,46.233433,115.271391,50.363929,115.543749,54.485277,115.0,58.573529
8,image_04.png,116.0,56.023810,115.136870,60.517880,113.524507,64.720681,112.105942,69.026299,110.226091,...,116.238526,37.755297,116.205878,42.348127,116.182206,46.961348,116.063657,51.573330,116.0,56.023810
9,image_05.png,116.0,45.016854,114.422396,48.463726,112.950592,52.137978,111.670189,55.913823,109.862924,...,110.697356,51.721988,111.971151,47.913293,114.712812,45.214738,115.892564,41.769587,116.0,45.016854


In [7]:
df_semilandmarks.to_csv("data/input_morphoJ_generated.csv", index=None)